In [1]:
%load_ext autoreload
%autoreload 2
# %matplotlib inline

import os
import polars as pl
import numpy as np
import spikeinterface.full as si
import matplotlib.pyplot as plt
# import IO_tools as io
from pathlib import Path
from pprint import pprint
from tools.settings import settings
from extract_sync_times import get_recording_sync

In [2]:
from spikeinterface import __version__ as sivers
print(f'spikeinterface version:  {sivers}')

spikeinterface version:  0.102.3


## setup

#### set paths

In [ ]:
# load config settings
paths = settings.paths
experiment_spiking = settings.experiment

# define project paths
raw_drive = paths.drive
experiment_name = experiment_spiking.dir
batch_folder = raw_drive / experiment_name / paths.data_dir
metadata_folder = raw_drive / experiment_name / paths.meta_dir
datasets_folder = raw_drive / experiment_name / paths.dataset_dir
print(f'Looking for recordings in:\n\t{batch_folder}')

# define session paths
raw_dir = paths.raw_dir
alignment_dir = paths.alignment_dir
processed_dir = paths.processed_dir
output_dir = paths.output_dir

Looking for recordings in:
	/mnt/array/3_TRAP_ISO/1_Recordings


#### set parallel processing parameters

In [4]:
job_kwargs=dict(
    n_jobs=8,
    chunk_duration='5s',
    progress_bar=True,
)

print("\nParallel Job parameters:")
pprint(job_kwargs, indent=2, width=2)


Parallel Job parameters:
{ 'chunk_duration': '5s',
  'n_jobs': 8,
  'progress_bar': True}


## process recordings

#### set subject / recording pairs to process

In [ ]:
# change to other than None to compress specific raw recording (including multiple segments)
# e.g. "NP02_R1", or keep as None to batch compress all recordings in recording_pairs
recording_name = None

recording_pairs = experiment_spiking.recordings
if recording_name is not None:
    properties = recording_pairs[recording_name]
    print(f'---processing single recording')
    recording_pairs = {recording_name: properties}

print("Processing the following recordings:")
pprint(recording_pairs, indent=4)

Processing the following recordings:
{'TRP804_R2': {'concatenate': False, 'multiple_shanks': True}}


#### load alignment data for experiment
the file should have been generated from `postprocess_recordings.ipynb`, and be located in the designated `alignment_dir` for the sessions [set by `default_config.toml`]. 

In [ ]:
# load alignment data
assert datasets_folder.exists(), f"(!) No datasets folder found for experiment: {experiment_spiking}\nExpected in: {datasets_folder}\nSkipping...\n\n"
if (alignment_file := next(datasets_folder.glob(f'{experiment_name}*_units_all_final.csv'), None)) is None:
    print(f'(!) No alignment data file found in:  {datasets_folder}\nSkipping...\n\n')
alignment_df = pl.read_csv(alignment_file)
print('Loaded alignment data:')
print('Total units:', alignment_df.height)
alignment_df.head(3)

Loaded alignment data:
Total units: 1050


,unit_id,brain_region_id,abbrev,unit_x,unit_y,unit_z,shank_id,channel_id,original_channel_idx,bregma,recording_name,brain_region,general_region,very_general_region,probe_id
i64,i64,f64,str,f64,f64,f64,i64,str,f64,str,str,str,str,str,i64
156,215,477.0,"""STR""",0.0,0.0,0.0,1,"""AP105""",124.0,"""[5739, 5400, 332]""","""TRP804_R2""","""Striatum""","""Striatum""","""Striatum""",0
167,233,477.0,"""STR""",282.502834,2652.801408,120.0,1,"""AP115""",129.0,"""[5739, 5400, 332]""","""TRP804_R2""","""Striatum""","""Striatum""","""Striatum""",0
165,230,477.0,"""STR""",282.500755,2656.795651,109.248516,1,"""AP115""",129.0,"""[5739, 5400, 332]""","""TRP804_R2""","""Striatum""","""Striatum""","""Striatum""",0


In [ ]:
overwrite = False

spiking_data = []
for session, properties in recording_pairs.items():
    animal = session.split('_')[0]
    recording_name = session
    concatenate = properties['concatenate']
    print(f'---processing  {recording_name}{", multiple recordings..." if concatenate else ""}')

    # find session folder
    rec_folder = batch_folder / animal / recording_name
    if rec_folder.exists():
        print(f'recording session folder:  {rec_folder}')  # top-level
    else:
        print(f'(!) No recording session folder found for:  {rec_folder}\nSkipping...\n\n')
        continue

    # check for multiple probes, essentially are there multiple .cbin/.bin files?
    raw_folder = rec_folder / raw_dir
    assert raw_folder.exists(), f"(!) No raw data folder found for recording: {rec_folder}\nExpected in: {raw_folder}\nSkipping...\n\n"
    match raw_files := list(raw_folder.rglob(f'{recording_name}*imec*.cbin')):
        case x if len(x) > 1:
            print(f'Found multiple raw files for {recording_name}: {x}')
        case _:
            print(f'Found single raw file for {recording_name}: {raw_files}')

    for probe_num, raw_file in enumerate(raw_files):
        print(f'---processing probe {probe_num} from file: {raw_file.name}')

        # get session metadata
        session_alignment = alignment_df.filter(
            pl.col('recording_name')==recording_name,  # match recording name
            pl.col('probe_id')==probe_num  # match probe number
            )
        if session_alignment.height == 0:
            print(f'(!) No units or alignment data found for {recording_name} probe {probe_num}\nSkipping...\n\n')
            continue

        # %% get sync times if not already present
        data_output = rec_folder / output_dir
        ping_samples, ping_times = get_recording_sync(
            raw_file=raw_file,
            rec_folder=rec_folder,
            probe_num=probe_num,
            overwrite=overwrite,
            threshold=40,
            sync_job_kwargs=dict(n_jobs=8, chunk_duration='10s', progress_bar=True)
        )
        if ping_samples is None: # type: ignore
            print(f'(!) No ping samples found in existing npy files for probe {probe_num}\nSkipping...\n\n')
            # continue
        
        # %% get spike times
        # load analyzer
        # spike_analyzer = load_spike_analyzer(rec_folder)
        processed_folder = rec_folder / processed_dir
        try:
            analyzer_folder = next(processed_folder.glob(f'*analyzer_clean_probe{probe_num}.zarr'))
        except StopIteration:
           analyzer_folder = next(processed_folder.glob(f'*analyzer_clean.zarr'), None)  # older format
        finally:
            assert analyzer_folder is not None, f"(!) No analyzer folder found for probe {probe_num}\nSkipping...\n\n"

        analyzer = si.load_sorting_analyzer(
            folder=analyzer_folder,
            format="zarr"
        )
        print(f'Loaded existing analyzer from "{analyzer_folder}"...')
        print(analyzer, '\n')

        # %% assigning metadata
        # general properties
        num_units = len(analyzer.unit_ids)
        analyzer.set_sorting_property('recording_name', [recording_name]*num_units, save=True)
        analyzer.set_sorting_property('animal',[animal]*num_units, save=True)
        analyzer.set_sorting_property('probe_id', [probe_num]*num_units, save=True)

        for prop in ['brain_region_id', 'abbrev', 'channel_id', 'brain_region', 'general_region']:
            analyzer.set_sorting_property(prop, session_alignment[prop].to_list(), save=False)        

        # %% extract spike times
        unit_to_channel_dict = {  # map unit ids to channel indices
            row[0]: row[1] 
            for row in session_alignment.select(['unit_id', 'original_channel_idx']).iter_rows()
        }
        spikes = analyzer.sorting.to_spike_vector(extremum_channel_inds=unit_to_channel_dict)
        spikes_df = pl.from_numpy(
            spikes[[ 'sample_index', 'unit_index']],
            schema={'sample_index': pl.Int32, 'unit_index': pl.Int32,}
        ).rename({'unit_index': 'unit_id'})

        # restrict to sync times (experimental period) in recording
        if ping_samples is None or len(ping_samples) < 2: # type: ignore
            print(f'(!) No sync timestamps found for probe {probe_num} in {recording_name}, keeping whole recording...\n\n')
            start_sync, end_sync = 0, analyzer.get_num_samples()  # use whole recording
        else:
            start_sync, end_sync = ping_samples[0], ping_samples[-2]
        spikes_df = spikes_df.filter(
            (pl.col('sample_index') >= start_sync) & (pl.col('sample_index') <= end_sync)
        )
        
        unit_data = spikes_df.group_by('unit_id').agg(
            pl.col('sample_index').alias('spike_times'),
            (pl.col('sample_index') / analyzer.sampling_frequency).alias('spike_times_sec'),
            pl.col('sample_index').count().alias('num_spikes'),
            firing_rate=(pl.col('sample_index').count() / (end_sync - start_sync)) * analyzer.sampling_frequency  # Hz
        )

        # merge unit data
        session_spiking = session_alignment.join(unit_data, on='unit_id', how='inner')

        # write to session datasets and add to experiment total
        session_spiking_file = data_output / f'{recording_name}_units_spiking_probe{probe_num}.parquet'
        session_spiking.write_parquet(session_spiking_file)
        print(f'Wrote spiking data for {session} probe {probe_num} to:\n\t{session_spiking_file}\n')
        spiking_data.append(session_spiking)

experiment_spiking = pl.concat(spiking_data, how='diagonal')
experiment_spiking.head(10)


### concatenate experiment firing

In [26]:
overwrite = False
session_datasets = list((raw_drive / experiment_name).rglob(f'*units_spiking_*.parquet'))
# session_datasets
spiking_data = []
for session_data_file in session_datasets:
    session_spiking = pl.scan_parquet(session_data_file).with_columns(
        animal=pl.lit(session_data_file.name.split('_')[0]),
    )
    recording_name = session_spiking.select('recording_name').first().collect()['recording_name'][0]
    probe_num = session_spiking.select('probe_id').first().collect()['probe_id'][0]
    print(f'Loaded spiking data for {recording_name} probe {probe_num} from:\n\t{session_data_file}\n')
    print(f'...concatenating data for recording:  {recording_name}')
    spiking_data.append(session_spiking)

experiment_spiking = pl.concat(spiking_data, how='diagonal')
experiment_spiking.collect()


Loaded spiking data for TRP803_R1 probe 0 from:
	/mnt/array/3_TRAP_ISO/1_Recordings/TRP803/TRP803_R1/3_datasets/TRP803_R1_units_spiking_probe0.parquet

...concatenating data for recording:  TRP803_R1
Loaded spiking data for TRP803_R2 probe 1 from:
	/mnt/array/3_TRAP_ISO/1_Recordings/TRP803/TRP803_R2/3_datasets/TRP803_R2_units_spiking_probe1.parquet

...concatenating data for recording:  TRP803_R2
Loaded spiking data for TRP801_R2 probe 0 from:
	/mnt/array/3_TRAP_ISO/1_Recordings/TRP801/TRP801_R2/3_datasets/TRP801_R2_units_spiking_probe0.parquet

...concatenating data for recording:  TRP801_R2
Loaded spiking data for TRP804_R1 probe 0 from:
	/mnt/array/3_TRAP_ISO/1_Recordings/TRP804/TRP804_R1/3_datasets/TRP804_R1_units_spiking_probe0.parquet

...concatenating data for recording:  TRP804_R1
Loaded spiking data for TRP804_R2 probe 0 from:
	/mnt/array/3_TRAP_ISO/1_Recordings/TRP804/TRP804_R2/3_datasets/TRP804_R2_units_spiking_probe0.parquet

...concatenating data for recording:  TRP804_R2


,unit_id,brain_region_id,abbrev,unit_x,unit_y,unit_z,shank_id,channel_id,original_channel_idx,bregma,recording_name,brain_region,general_region,very_general_region,probe_id,spike_times,spike_times_sec,num_spikes,firing_rate,animal
i64,i64,f64,str,f64,f64,f64,i64,str,f64,str,str,str,str,str,i64,list[i32],list[f64],u32,f64,str
9,18,952.0,"""EPd""",0.0,205.014458,116.518117,0,"""AP26""",26.0,"""[5739, 5400, 332]""","""TRP803_R1""","""Endopiriform nucleus, dorsal p…","""Cortical subplate""","""Cortical subplate""",0,"[1813967, 1814980, … 182557847]","[60.506215, 60.540005, … 6089.352442]",83998,13.932278,"""TRP803"""
99,146,672.0,"""CP""",0.0,0.0,0.0,0,"""AP375""",283.0,"""[5739, 5400, 332]""","""TRP803_R1""","""Caudoputamen""","""Striatum""","""Striatum""",0,"[1813213, 1814812, … 182552800]","[60.481065, 60.534401, … 6089.184095]",126754,21.023977,"""TRP803"""
14,24,952.0,"""EPd""",0.0,253.731895,110.051909,0,"""AP34""",34.0,"""[5739, 5400, 332]""","""TRP803_R1""","""Endopiriform nucleus, dorsal p…","""Cortical subplate""","""Cortical subplate""",0,"[1815293, 1819776, … 182561559]","[60.550445, 60.699979, … 6089.476258]",98197,16.287387,"""TRP803"""
2,3,952.0,"""EPd""",0.0,41.27799,111.197747,0,"""AP6""",6.0,"""[5739, 5400, 332]""","""TRP803_R1""","""Endopiriform nucleus, dorsal p…","""Cortical subplate""","""Cortical subplate""",0,"[1814755, 1815948, … 182551894]","[60.5325, 60.572293, … 6089.153875]",46802,7.762786,"""TRP803"""
7,15,952.0,"""EPd""",0.0,205.0,0.0,0,"""AP22""",22.0,"""[5739, 5400, 332]""","""TRP803_R1""","""Endopiriform nucleus, dorsal p…","""Cortical subplate""","""Cortical subplate""",0,"[1821050, 1823851, … 182559754]","[60.742474, 60.835903, … 6089.416051]",26630,4.416969,"""TRP803"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
73,110,1022.0,"""GPe""",0.0,2461.509751,111.715518,0,"""AP86""",35.0,"""[5739, 5400, 332]""","""TRP804_R2""","""Globus pallidus, external segm…","""Pallidum""","""Pallidum""",0,"[1078, 10196, … 187390730]","[0.035957, 0.340095, … 6250.55684]",23050,3.687503,"""TRP804"""
85,131,966.0,"""EPv""",250.0,1010.0,14.248032,1,"""AP284""",150.0,"""[5739, 5400, 332]""","""TRP804_R2""","""Endopiriform nucleus, ventral …","""Cortical subplate""","""Cortical subplate""",0,"[509, 1589, … 187394338]","[0.016978, 0.053002, … 6250.677188]",46334,7.41244,"""TRP804"""
70,104,1022.0,"""GPe""",0.0,2480.918602,57.394618,0,"""AP86""",35.0,"""[5739, 5400, 332]""","""TRP804_R2""","""Globus pallidus, external segm…","""Pallidum""","""Pallidum""",0,"[919, 1800, … 187392252]","[0.030654, 0.06004, … 6250.607608]",37114,5.937439,"""TRP804"""


In [31]:
experiment_spiking.unique('brain_region').collect()['brain_region'].to_list()

['Superior colliculus, superficial gray layer',
 'Cuneiform nucleus',
 'amygdalar capsule',
 'Basolateral amygdalar nucleus, ventral part',
 'Superior colliculus, motor related, deep gray layer',
 'Globus pallidus, internal segment',
 'Endopiriform nucleus, dorsal part',
 'Ventral posterolateral nucleus of the thalamus',
 'Pons',
 'corpus callosum, body',
 'Primary somatosensory area, upper limb, layer 6a',
 'Retrosplenial area, ventral part, layer 5',
 'Superior colliculus, motor related, intermediate white layer',
 'Basolateral amygdalar nucleus, posterior part',
 'supra-callosal cerebral white matter',
 'Caudoputamen',
 'Retrosplenial area, ventral part, layer 1',
 'Parabrachial nucleus',
 'Basolateral amygdalar nucleus, anterior part',
 'Primary somatosensory area, unassigned, layer 6a',
 'Piriform-amygdalar area',
 'external capsule',
 'Cortical subplate',
 'Retrosplenial area, dorsal part, layer 2/3',
 'Globus pallidus, external segment',
 'Central amygdalar nucleus, capsular par

In [47]:
# clean up total dataset
region_remove_list = [
    'root', 'corpus callosum', 'supra-callosal cerebral white matter', 'internal capsule', 'external capsule', 'white matter'
]
final_experiment_spiking = experiment_spiking.collect()
print(f'Nulls: {final_experiment_spiking.filter(pl.col("brain_region").is_null()).height}')
print(f'Unassigned or null brain regions: {final_experiment_spiking.filter(pl.col("brain_region").is_in(region_remove_list)).height}')

final_experiment_spiking = (
    final_experiment_spiking
    .filter(
        ~pl.col('brain_region').is_in(region_remove_list))
    .drop_nulls(subset=['brain_region'])
    .select([
        'animal', 'recording_name',
        'unit_id', 'brain_region_id', 'abbrev', 'brain_region', 'general_region', 
        'probe_id', 'channel_id', 'original_channel_idx',
        'unit_x', 'unit_y', 'unit_z', 'bregma',
        'num_spikes', 'spike_times', 'spike_times_sec', 'firing_rate'])
)

print(f'Total removed units: {experiment_spiking.collect().height - final_experiment_spiking.height}')
print(f'Total remaining units: {final_experiment_spiking.height}')
final_experiment_spiking

Nulls: 7
Unassigned or null brain regions: 15
Total removed units: 22
Total remaining units: 552


animal,recording_name,unit_id,brain_region_id,abbrev,brain_region,general_region,probe_id,channel_id,original_channel_idx,unit_x,unit_y,unit_z,bregma,num_spikes,spike_times,spike_times_sec,firing_rate
str,str,i64,f64,str,str,str,i64,str,f64,f64,f64,f64,str,u32,list[i32],list[f64],f64
"""TRP803""","""TRP803_R1""",18,952.0,"""EPd""","""Endopiriform nucleus, dorsal p…","""Cortical subplate""",0,"""AP26""",26.0,0.0,205.014458,116.518117,"""[5739, 5400, 332]""",83998,"[1813967, 1814980, … 182557847]","[60.506215, 60.540005, … 6089.352442]",13.932278
"""TRP803""","""TRP803_R1""",146,672.0,"""CP""","""Caudoputamen""","""Striatum""",0,"""AP375""",283.0,0.0,0.0,0.0,"""[5739, 5400, 332]""",126754,"[1813213, 1814812, … 182552800]","[60.481065, 60.534401, … 6089.184095]",21.023977
"""TRP803""","""TRP803_R1""",24,952.0,"""EPd""","""Endopiriform nucleus, dorsal p…","""Cortical subplate""",0,"""AP34""",34.0,0.0,253.731895,110.051909,"""[5739, 5400, 332]""",98197,"[1815293, 1819776, … 182561559]","[60.550445, 60.699979, … 6089.476258]",16.287387
"""TRP803""","""TRP803_R1""",3,952.0,"""EPd""","""Endopiriform nucleus, dorsal p…","""Cortical subplate""",0,"""AP6""",6.0,0.0,41.27799,111.197747,"""[5739, 5400, 332]""",46802,"[1814755, 1815948, … 182551894]","[60.5325, 60.572293, … 6089.153875]",7.762786
"""TRP803""","""TRP803_R1""",15,952.0,"""EPd""","""Endopiriform nucleus, dorsal p…","""Cortical subplate""",0,"""AP22""",22.0,0.0,205.0,0.0,"""[5739, 5400, 332]""",26630,"[1821050, 1823851, … 182559754]","[60.742474, 60.835903, … 6089.416051]",4.416969
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""TRP804""","""TRP804_R2""",110,1022.0,"""GPe""","""Globus pallidus, external segm…","""Pallidum""",0,"""AP86""",35.0,0.0,2461.509751,111.715518,"""[5739, 5400, 332]""",23050,"[1078, 10196, … 187390730]","[0.035957, 0.340095, … 6250.55684]",3.687503
"""TRP804""","""TRP804_R2""",131,966.0,"""EPv""","""Endopiriform nucleus, ventral …","""Cortical subplate""",0,"""AP284""",150.0,250.0,1010.0,14.248032,"""[5739, 5400, 332]""",46334,"[509, 1589, … 187394338]","[0.016978, 0.053002, … 6250.677188]",7.41244
"""TRP804""","""TRP804_R2""",104,1022.0,"""GPe""","""Globus pallidus, external segm…","""Pallidum""",0,"""AP86""",35.0,0.0,2480.918602,57.394618,"""[5739, 5400, 332]""",37114,"[919, 1800, … 187392252]","[0.030654, 0.06004, … 6250.607608]",5.937439


In [48]:
# saving total
final_spiking_file = datasets_folder / f'{experiment_name}_units_spiking_all.parquet'
final_experiment_spiking.write_parquet(final_spiking_file)
